## Обновление тарифов в OCS
- Выполняется проверка изменений в тарифах относительно текущих
- Готовятся файлы для загрузки изменений в OCS DB

In [19]:
import numpy as np
import pandas as pd
import pyodbc
from pandas import Series, DataFrame
from os.path import join

#######################################################################################
# Define variables here (Part 1/3)
#######################################################################################

downloads = r'c:\Users\balob\Downloads\new'
file_allowed_list = 'DMI.dmi_allowed_list_export_Thu_May_14_2020.csv'
eur_usd_rate = 1.15

#######################################################################################

#######################################################################################

In [10]:
# Из базы данных OCS выгружаем TADIG и все Price Plans

# Connect to OCSDBREP1 (BSS)

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

# Получение данных TADIG

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME AS COUNTRY,n.NETWORK_NAME AS NETWORK,t.TADIG_CODE AS TADIG,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''
df_tadig = pd.read_sql_query(sql_srt, cnxn)

print('\nOCS TADIG:')
display(df_tadig.head(3))

# Получение тарифов Partner, P4, MB, Smart из OCS по прайспланам 267, 268, 343, 322, 323, 324

# 267 Partner Cost Plan 2018
# 321 Partner Charge 2019
# 268 P4 Cost Plan 2018
# 322 P4 Charge 2019
# 329 MB Cost Plan 2019
# 330 MB Charge 2019
# 343 SMART Cost Plan 2019
# 344 SMART Charge 2019
# 372 S8 Cost 2019
# 373 S8 Charge 2019
# 323 Full Combined Charge 2019
# 324 Combined Charge 2019

sql_srt=\
'''
SELECT * FROM ROAMING_PLAN_RULES
WHERE roaming_plan_id in (267, 321, 268, 322, 329, 330, 343, 344, 323, 324, 372, 373, 368)
AND end_date IS NULL
'''
df_ocs_all = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()

print('\nOCS all tariffs:')
display(df_ocs_all.head(3))


OCS TADIG:


,COUNTRY,NETWORK,TADIG,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0



OCS all tariffs:


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN,DailyCap,DailyCapLimit,DailyCapTimeZone
0,5567.0,267.0,59.0,1.0,0.990086,0.990086,0.249622,0.0,10.238187,1.0,2017-11-01,None,False,None,False,NaN,NaN
1,5568.0,267.0,95.0,1.0,2.189060,2.189060,0.450132,0.0,14.333252,1.0,2017-11-01,None,False,None,False,NaN,NaN
2,5571.0,267.0,113.0,1.0,2.367968,0.000000,0.494058,0.0,15.786522,1.0,2017-11-01,None,False,None,False,NaN,NaN


In [20]:
# Описываем класс данных Спонсора и создаем инстансы для Спонсоров

class SponsorClass:
    
#######################################################################################
# Define variables here (Part 2/3)
#######################################################################################
    
    downloads = downloads
    eur_usd_rate = eur_usd_rate
    tariff_tolerance = 0.0001 # Absolute tolerance
    smart_price_to_cost_rate = 1.15
    discount_data_price_threshold = 1 # USD Порог для выборки из файла
    charge_increment = 2

#######################################################################################

#######################################################################################

    col_list = [
        'ROAMING_PLAN_RULE_ID',
        'ROAMING_PLAN_ID',
        'NETWORK_ID',
        'SPONSOR_ID',
        'MOC_NEW',
        'MTC_NEW',
        'SMS_NEW',
        'MT_SMS_RATE',
        'DATA_NEW',
        'CURRENCY_ID',
        'START_DATE',
        'END_DATE',
        'is_discounted',
        'HIDDEN',
        'DailyCap_NEW',
        'DailyCapLimit_NEW']

    def __init__(self, name, sponsor_id, ocs_roaming_plan_id, tariff_file_name,
                 applied_from, ocs_charge_roaming_plan_id_list, al_combined):
        self.name = name
        self.sponsor_id = sponsor_id
        self.ocs_roaming_plan_id = ocs_roaming_plan_id
        self.tariff_file_name = tariff_file_name
        self.applied_from = applied_from
        self.ocs_charge_roaming_plan_id_list = ocs_charge_roaming_plan_id_list
        self.al_combined = al_combined
        self.df_new_tariff = self.upload_data_from_file(df_tadig)
        self.df_ocs_tariff = self.upload_data_from_ocs()
        self.df_ocs_and_new_tariff = self.merge_new_and_ocs_tariffs()
    
    def upload_data_from_ocs(self): # self.df_ocs_tariff
        '''Выгрузка данных из OCS'''
        sql_srt=\
        '''
        SELECT
        MO_CALL_RATE MOC,
        MT_CALL_RATE MTC,
        MO_SMS_RATE SMS,
        DATA_RATE DATA,
        NETWORK_ID,SPONSOR_ID,is_discounted,DailyCap,DailyCapLimit
        FROM ROAMING_PLAN_RULES WHERE roaming_plan_id = {0} AND SPONSOR_ID = {1}
        AND end_date IS NULL
        '''.format(self.ocs_roaming_plan_id, self.sponsor_id)
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')
        df_temp = pd.read_sql_query(sql_srt, cnxn)
        cnxn.close()
        return df_temp
        
    def upload_data_from_file(self, df_tadig): # self.df_new_tariff
        '''Загрузка новых тарифов из файла.
        Выполняется добавление NETWORK_ID из OCS по TADIG.
        Выполняется фильтрация по максимальному тарифу передачи данных discount_data_price_threshold.'''
        df_temp = pd.read_csv(join(self.downloads, self.tariff_file_name))
        df_temp = df_temp[df_temp['DATA'] <= self.discount_data_price_threshold]
        df_temp = pd.merge(df_temp, df_tadig[['TADIG','NETWORK_ID']], how='left', on='TADIG')
        df_temp.DailyCap.fillna(0, inplace=True)
        return df_temp
    
    def merge_new_and_ocs_tariffs(self): # self.df_ocs_and_new_tariff
        '''Объединить новые и текущие тарифы в один DataFrame'''
        return pd.merge(self.df_new_tariff, self.df_ocs_tariff, how='outer', on='NETWORK_ID', suffixes=['_NEW', '_CURRENT'])
    
    def new_tariffs(self):
        '''Функция возвращает DF с появившимися тарифами в новых тарифах Спонсора'''
        cols = ['TADIG' ,'MOC_NEW','MTC_NEW','SMS_NEW','DATA_NEW','SPONSOR_ID','LTE','CAMEL','NETWORK_ID',
                'DailyCap_NEW','DailyCapLimit_NEW','is_discounted']
        return self.df_ocs_and_new_tariff.loc[(self.df_ocs_and_new_tariff.DATA_NEW.notnull())
                                              & (self.df_ocs_and_new_tariff.SPONSOR_ID.isnull()), cols].copy()
    
    def closed_tariffs(self, df_al_comb, df_tadig):
        '''Функция возвращает DF с пропавшими тарифами в новых тарифах Спонсора'''
        cols = ['NETWORK_ID' ,'MOC_CURRENT','MTC_CURRENT','SMS_CURRENT','DATA_CURRENT']
        df_temp = self.df_ocs_and_new_tariff.loc[(self.df_ocs_and_new_tariff.DATA_NEW.isnull())
                                              & (self.df_ocs_and_new_tariff.SPONSOR_ID.notnull())
                                              & (self.df_ocs_and_new_tariff.is_discounted == True), cols].copy()
        df_temp = pd.merge(df_temp, df_al_comb, how='left', on='NETWORK_ID')
        df_temp.rename(
            {'MOC_CURRENT':'MOC', 'MTC_CURRENT':'MTC', 'SMS_CURRENT':'SMS', 'DATA_CURRENT':'DATA', 'SPONSOR':'SPONSOR_COMB'},
                       axis=1, inplace=True)
        cols = ['COUNTRY','NETWORK', 'TADIG','MOC', 'MTC', 'SMS', 'DATA', 'NETWORK_ID', 'SPONSOR_COMB']
        return pd.merge(df_temp, df_tadig, how='left', on='NETWORK_ID')[cols]
    
    def changes_in_tariffs(self):
        '''Функция возвращает DF с изменившимися тарифами в новых тарифах Спонсора
        по сравнению с текущими тарифами OCS'''
        return self.df_ocs_and_new_tariff.loc[(self.df_ocs_and_new_tariff.DATA_NEW.notnull()) &
                                              (self.df_ocs_and_new_tariff.DATA_CURRENT.notnull()) &
                                              (\
             ~np.isclose(self.df_ocs_and_new_tariff['MOC_NEW'], self.df_ocs_and_new_tariff['MOC_CURRENT'],
                         atol = self.tariff_tolerance, equal_nan = True) |\
             ~np.isclose(self.df_ocs_and_new_tariff['MTC_NEW'], self.df_ocs_and_new_tariff['MTC_CURRENT'],
                         atol = self.tariff_tolerance, equal_nan = True) |\
             ~np.isclose(self.df_ocs_and_new_tariff['SMS_NEW'], self.df_ocs_and_new_tariff['SMS_CURRENT'],
                         atol = self.tariff_tolerance, equal_nan = True) |\
             ~np.isclose(self.df_ocs_and_new_tariff['DATA_NEW'], self.df_ocs_and_new_tariff['DATA_CURRENT'],
                         atol = self.tariff_tolerance, equal_nan = True))].copy()        
        
    def changes_in_tariffs_colorised(self):
        '''Функция ищет изменившиеся тарифы в новых тарифах Спонсора'''       
        df_diff = self.changes_in_tariffs()
        def colorize(s):
            '''
            highlight the maximum in a Series yellow.
            '''
            color_list = []
            for tariff_type in [['MOC_CURRENT','MOC_NEW'], ['MTC_CURRENT','MTC_NEW'], ['SMS_CURRENT','SMS_NEW'], ['DATA_CURRENT','DATA_NEW']]:
                if np.isclose(s[tariff_type[0]], s[tariff_type[1]], atol = self.tariff_tolerance, equal_nan=True):
                    color_list.extend(['', ''])
                elif s[tariff_type[0]] > s[tariff_type[1]]:
                    color_list.extend(['background-color: yellow', 'background-color: #81EF21'])
                else:
                    color_list.extend(['background-color: yellow', 'background-color: red'])
            return color_list
        cols = ['TADIG','MOC_NEW','MOC_CURRENT','MTC_NEW','MTC_CURRENT','SMS_NEW','SMS_CURRENT','DATA_NEW','DATA_CURRENT'
                   ,'is_discounted','NETWORK_ID']
        return df_diff[cols].style.apply(colorize, subset=['MOC_CURRENT','MOC_NEW','MTC_CURRENT','MTC_NEW','SMS_CURRENT','SMS_NEW','DATA_CURRENT','DATA_NEW'], axis=1)

    def prepare_file_to_close_tariffs(self):
        '''Подготовить файлы для закрытия тарифов в OCS'''
        df_closed = self.closed_tariffs(df_al_comb, df_tadig)
        df_diff = self.changes_in_tariffs()
        df_ocs_close = df_ocs_all[
            (df_ocs_all.NETWORK_ID.isin(df_closed.NETWORK_ID.values)    # Тарифы для закрытия
             | (df_ocs_all.NETWORK_ID.isin(df_diff.NETWORK_ID.values))) # Тарифы с изменением
            & (df_ocs_all.SPONSOR_ID == self.sponsor_id)].drop_duplicates()
        df_ocs_close.to_csv(join(self.downloads, f'file_to_db_{self.name}_close.csv'), index=False)
        return None
    
    def prepare_file_to_add_different(self):
        '''Подготовить файлы с различающимися тарифами для загрузки тарифов OCS'''
        df_diff = self.changes_in_tariffs()
        df_diff['ROAMING_PLAN_RULE_ID'] = ''
        df_diff['SPONSOR_ID'] = self.sponsor_id
        df_diff['MT_SMS_RATE'] = 0
        df_diff['CURRENCY_ID'] = 1 # 1:USD, 2:EUR
        df_diff['START_DATE'] = self.applied_from
        df_diff['END_DATE'] = 'NULL'
        df_diff['HIDDEN'] = 0
        
        df_diff_100 = df_diff.copy()
        col_prices_list = ['MOC_NEW','MTC_NEW','SMS_NEW','DATA_NEW']
        df_diff_100[col_prices_list] = df_diff_100[col_prices_list] * self.charge_increment
        
        df_diff['ROAMING_PLAN_ID'] = self.ocs_roaming_plan_id

        df_diff[self.col_list].drop_duplicates()\
        .to_csv(join(self.downloads, f'file_to_db_{self.name}_diff.csv'), index=False)
        
        df_diff_100_all = DataFrame()
        
        for roaming_plan_id in self.ocs_charge_roaming_plan_id_list:
            df_temp = df_diff_100.copy()
            df_temp['ROAMING_PLAN_ID'] = roaming_plan_id
            df_diff_100_all = df_diff_100_all.append(df_temp)
            
        df_diff_100_all[self.col_list].drop_duplicates().\
        to_csv(join(self.downloads, f'file_to_db_{self.name}_diff_100.csv'), index=False)
        
    def prepare_file_to_add_new(self):
        '''Подготовить файлы с новыми тарифами для загрузки тарифов в OCS'''
        df_new = self.new_tariffs().copy()
        df_new['ROAMING_PLAN_RULE_ID'] = ''
        df_new['SPONSOR_ID'] = self.sponsor_id
        df_new['MT_SMS_RATE'] = 0
        df_new['CURRENCY_ID'] = 1 # 1:USD, 2:EUR
        df_new['START_DATE'] = self.applied_from
        df_new['END_DATE'] = 'NULL'
        df_new['HIDDEN'] = 0

        df_new_100 = df_new.copy()
        col_prices_list = ['MOC_NEW','MTC_NEW','SMS_NEW','DATA_NEW']
        df_new_100[col_prices_list] = df_new_100[col_prices_list] * self.charge_increment
        
        df_new['ROAMING_PLAN_ID'] = self.ocs_roaming_plan_id

        df_new[self.col_list].drop_duplicates().to_csv(join(self.downloads, f'file_to_db_{self.name}_new.csv'), index=False)
        
        df_new_100_all = DataFrame()
        
        for roaming_plan_id in self.ocs_charge_roaming_plan_id_list:
            df_temp = df_new_100.copy()
            df_temp['ROAMING_PLAN_ID'] = roaming_plan_id
            df_new_100_all = df_new_100_all.append(df_temp)
            
        df_new_100_all[self.col_list].drop_duplicates().\
        to_csv(join(self.downloads, f'file_to_db_{self.name}_new_100.csv'), index=False)
        
#######################################################################################
# Define variables here (Part 3/3)
#######################################################################################

# Задать имена для существующих, основных "Combined" Allowed list
s1_al_combined_def_name = 'Partner_Combined'
s2_al_combined_def_name = 'P4_Combined'
s4_al_combined_def_name = 'MB_Combined'
s5_al_combined_def_name = 'SMART_Combined'
s6_al_combined_def_name = 'S6_Combined'
s8_al_combined_def_name = 'S8_Combined'

# Files with the tariffs should contain the following fields:
# TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCapLimit

# SponsorClass(name, sponsor_id, ocs_roaming_plan_id, tariff_file_name, applied_from,
# ocs_charge_roaming_plan_id_list, al_combined)

s1_sponsor = SponsorClass('S1', 1, 267, 'S1_20200301_FULL.csv', '2020-01-22 00:00:00', [321, 323, 324],
                          s1_al_combined_def_name)

s2_sponsor = SponsorClass('S2', 2, 268, 'S2_20200601_FULL_USD.csv', '2020-06-01 00:00:00', [322, 323, 324],
                          s2_al_combined_def_name)

s4_sponsor = SponsorClass('S4', 4, 329, 'S4_20200304_FULL.csv', '2020-03-04 00:00:00', [330, 323, 324],
                          s4_al_combined_def_name)

s5_sponsor = SponsorClass('S5', 5, 343, 'S5_20200101_FULL_RB_COST.csv', '2020-01-22 00:00:00', [344, 323, 324],
                          s5_al_combined_def_name)

s8_sponsor = SponsorClass('S8', 8, 372, 'S8_20200101_FULL.csv', '2020-01-22 00:00:00', [373, 323, 324],
                          s8_al_combined_def_name)

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s8_sponsor]

#######################################################################################

#######################################################################################

In [21]:
# Allowed lists
df_al_comb = pd.read_csv(join(downloads, file_allowed_list), usecols=['Sponsor','PLMN code'])
df_al_comb.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'}, inplace=True,axis=1)
df_al_comb = df_al_comb[df_al_comb.SPONSOR.isin([sponsor.al_combined for sponsor in sponsors])]
df_al_comb.replace(dict([sponsor.al_combined, sponsor.name] for sponsor in sponsors), inplace=True)
df_al_comb = pd.merge(df_al_comb, df_tadig[['TADIG','NETWORK_ID']], how='left',
                 left_on='TADIG_CODE_AL', right_on='TADIG')
df_al_comb.drop('TADIG', inplace=True, axis=1)
df_al_comb.head()

,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,S2,HRVT2,225.0
1,S2,CZEET,234.0
2,S1,GBRAJ,998.0
3,S2,HRVVI,226.0
4,S1,GBRHU,1007.0


In [22]:
# Текущие тарифы OCS

for sponsor in sponsors:
    print(f'\nOCS tariffs of {sponsor.name}:')
    display(sponsor.df_ocs_tariff.head(3))


OCS tariffs of S1:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted,DailyCap,DailyCapLimit
0,0.26,0.0,0.030,0.0090,9.0,1.0,True,False,None
1,0.03,0.0,0.006,0.0024,12.0,1.0,True,False,None
2,0.09,0.0,0.030,0.0300,17.0,1.0,False,False,None



OCS tariffs of S2:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted,DailyCap,DailyCapLimit
0,0.51635,0.000,0.059800,0.04600,9.0,2.0,True,False,None
1,0.44850,0.000,0.046000,0.04485,10.0,2.0,True,False,None
2,0.99900,0.999,0.290905,9.00000,25.0,2.0,False,False,None



OCS tariffs of S4:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted,DailyCap,DailyCapLimit
0,0.26,0.26,0.2600,0.005,49.0,4.0,True,False,None
1,0.26,0.26,0.2600,0.005,164.0,4.0,True,False,None
2,NaN,0.00,0.0948,NaN,196.0,4.0,False,False,None



OCS tariffs of S5:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted,DailyCap,DailyCapLimit
0,NaN,0.1725,0.115000,0.115000,4.0,5.0,True,False,NaN
1,NaN,0.0000,0.017250,0.017250,6.0,5.0,True,False,NaN
2,0.161,0.0000,0.008625,0.008625,12.0,5.0,True,False,NaN



OCS tariffs of S8:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted,DailyCap,DailyCapLimit
0,1.0164,0.0,0.1236,0.1044,5.0,8.0,True,False,None
1,0.7956,0.0,0.2880,0.2976,6.0,8.0,True,False,None
2,0.6096,0.0,0.1932,0.0180,9.0,8.0,True,False,None


In [23]:
# Новые тарифы из файлов

for sponsor in sponsors:
    print(f'\nNew tariffs of {sponsor.name}:')
    display(sponsor.df_new_tariff.head(3))


New tariffs of S1:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID
0,ALBVF,0.0300,0.0,0.0060,0.0024,1,1.0,Yes,Yes,0.0,NaN,12.0
1,ANDMA,0.0900,0.0,0.0300,0.0300,1,1.0,No,No,0.0,NaN,17.0
2,AIACW,0.1606,0.0,0.0128,0.0257,60,1.0,No,Yes,0.0,NaN,20.0



New tariffs of S2:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID
0,ALBAM,0.51635,0,0.05980,0.04600,1.0,1,Yes,Yes,0.0,NaN,9.0
1,ALBEM,0.44850,0,0.04600,0.04485,60.0,1,Yes,Yes,0.0,NaN,10.0
2,ARGTM,NaN,0,0.16445,0.04600,NaN,1,No,Yes,0.0,NaN,27.0



New tariffs of S4:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID
0,HKGH3,0.013,0.013,0.13,0.00315,60,1,Yes,60,0.0,NaN,392.0
1,HKGHT,0.013,0.013,0.13,0.00315,60,1,Yes,60,0.0,NaN,392.0
2,MACHT,0.130,0.130,0.13,0.00504,60,1,Yes,60,0.0,NaN,567.0



New tariffs of S5:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID
0,AFGAW,NaN,0.1725,0.115000,0.115000,60,1.0,No,NaN,0.0,NaN,4.0
1,AFGEA,NaN,0.0000,0.017250,0.017250,60,1.0,No,NaN,0.0,NaN,6.0
2,ALBVF,0.161,0.0000,0.008625,0.008625,60,1.0,No,NaN,0.0,NaN,12.0



New tariffs of S8:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID
0,ABWDC,0.3576,0.0,0.1800,0.0720,NaN,1,No,Yes,0.0,NaN,1160.0
1,AFGAR,1.0164,0.0,0.1236,0.1044,NaN,1,No,No,0.0,NaN,5.0
2,AFGEA,0.7956,0.0,0.2880,0.2976,NaN,1,No,Yes,0.0,NaN,6.0


In [24]:
# Отсутствующие в BD OCS TADIG в новых тарифах

for sponsor in sponsors:
    print(f'\nОтсутствующие в BD OCS TADIG в новых тарифах {sponsor.name}:')
    display(sponsor.df_new_tariff[sponsor.df_new_tariff.NETWORK_ID.isnull()])


Отсутствующие в BD OCS TADIG в новых тарифах S1:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID



Отсутствующие в BD OCS TADIG в новых тарифах S2:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID
119,K0001,0.52900,0,0.013225,0.066125,1.0,1,Yes,Yes,0.0,NaN,NaN
138,MNGSK,0.40595,0,0.059685,0.059685,60.0,1,No,Yes,0.0,NaN,NaN
142,MOZVG,0.80500,0,0.080500,0.048300,60.0,10,No,Yes,0.0,NaN,NaN
144,MMRVG,0.57684,0,0.059685,0.358225,60.0,10,Yes,Yes,0.0,NaN,NaN



Отсутствующие в BD OCS TADIG в новых тарифах S4:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID



Отсутствующие в BD OCS TADIG в новых тарифах S5:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID
3,AIAK9,0.5750,0.0575,0.05750,0.05750,60,1.0,No,NaN,0.0,NaN,NaN
5,ATGK9,0.5750,0.0575,0.05750,0.05750,60,1.0,No,NaN,0.0,NaN,NaN
29,BMUK9,0.2875,0.0345,0.02300,0.01150,60,1.0,No,NaN,0.0,NaN,NaN
32,BOLK9,NaN,0.0575,0.01150,0.11500,60,1.0,No,NaN,0.0,NaN,NaN
45,BFAK8,NaN,0.0000,NaN,0.01150,60,1.0,No,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
342,VUTK8,0.5750,0.0000,0.01150,0.01150,60,1.0,No,NaN,0.0,NaN,NaN
345,ZMBK8,NaN,0.0000,NaN,0.11500,60,1.0,No,NaN,0.0,NaN,NaN
346,CAFK9,0.3910,0.0000,0.01288,0.01288,60,1.0,No,NaN,0.0,NaN,NaN
347,GINK9,0.3910,0.0000,0.01288,0.01288,60,1.0,No,NaN,0.0,NaN,NaN



Отсутствующие в BD OCS TADIG в новых тарифах S8:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID
23,BDIL1,0.8928,0.0000,0.2700,0.3552,NaN,1,No,No,0.0,NaN,NaN
33,BHSNC,1.3368,0.0000,0.1980,0.2136,NaN,1,Yes,Yes,0.0,NaN,NaN
38,BLZSC,0.8928,0.0900,0.1800,0.0900,NaN,1,Yes,Yes,0.0,NaN,NaN
60,CHNCM,0.4992,0.2496,0.1236,0.0144,NaN,1,Yes,No,0.0,NaN,NaN
124,GINAG,1.0164,0.0000,0.1236,0.1044,NaN,1,No,No,0.0,NaN,NaN
126,GNQHT,0.8028,0.0000,0.2700,0.2676,NaN,1,No,Yes,0.0,NaN,NaN
192,KIRKL,4.9632,0.0000,0.2544,0.1644,NaN,1,No,No,0.0,NaN,NaN
215,MCOM1,1.0164,0.0000,0.1332,0.2052,NaN,1,No,No,0.0,NaN,NaN
221,MEXN3,0.3576,0.0900,0.2976,0.3552,NaN,1,No,No,0.0,NaN,NaN
234,MOZVG,0.9804,0.0000,0.1980,0.1776,NaN,1,No,Yes,0.0,NaN,NaN


In [25]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

for sponsor in sponsors:
    print(f'\nРазличные тарифы для одного и того же оператора в тарифах {sponsor.name}:')
    df_temp = sponsor.df_new_tariff.groupby('NETWORK_ID')['MOC','MTC','SMS','DATA'].nunique()
    s1 = df_temp[df_temp.values > 1]
    display(s1)


Различные тарифы для одного и того же оператора в тарифах S1:


,MOC,MTC,SMS,DATA
NETWORK_ID,,,,



Различные тарифы для одного и того же оператора в тарифах S2:


,MOC,MTC,SMS,DATA
NETWORK_ID,,,,



Различные тарифы для одного и того же оператора в тарифах S4:


,MOC,MTC,SMS,DATA
NETWORK_ID,,,,



Различные тарифы для одного и того же оператора в тарифах S5:


,MOC,MTC,SMS,DATA
NETWORK_ID,,,,



Различные тарифы для одного и того же оператора в тарифах S8:


,MOC,MTC,SMS,DATA
NETWORK_ID,,,,


In [18]:
s2_sponsor.df_new_tariff[s2_sponsor.df_new_tariff.NETWORK_ID == 412]

,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,NETWORK_ID
77,IND10,0.4600,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0
78,IND11,0.4715,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0
79,IND12,0.4600,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0
80,IND14,0.4715,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0
81,IND15,0.4715,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0
82,IND16,0.4600,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0
83,INDA1,0.4715,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0
84,INDA2,0.4715,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0
85,INDA3,0.4715,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0
86,INDA4,0.4715,0.0,0.161,0.046,60.0,1,Yes,No,0.0,NaN,412.0


In [26]:
# Вывести появившиеся тарифы с выгрузкой в файлы

for sponsor in sponsors:
    print(f'\nПоявившиеся тарифы {sponsor.name}:')
    sponsor.new_tariffs().to_csv(join(sponsor.downloads, f'new_tadig_{sponsor.name}.csv'), index=False)
    display(sponsor.new_tariffs())


Появившиеся тарифы S1:


,TADIG,MOC_NEW,MTC_NEW,SMS_NEW,DATA_NEW,SPONSOR_ID,LTE,CAMEL,NETWORK_ID,DailyCap_NEW,DailyCapLimit_NEW,is_discounted



Появившиеся тарифы S2:


,TADIG,MOC_NEW,MTC_NEW,SMS_NEW,DATA_NEW,SPONSOR_ID,LTE,CAMEL,NETWORK_ID,DailyCap_NEW,DailyCapLimit_NEW,is_discounted
119,K0001,0.52900,0.0,0.013225,0.066125,NaN,Yes,Yes,NaN,0.0,NaN,NaN
120,MNGSK,0.40595,0.0,0.059685,0.059685,NaN,No,Yes,NaN,0.0,NaN,NaN
121,MOZVG,0.80500,0.0,0.080500,0.048300,NaN,No,Yes,NaN,0.0,NaN,NaN
122,MMRVG,0.57684,0.0,0.059685,0.358225,NaN,Yes,Yes,NaN,0.0,NaN,NaN



Появившиеся тарифы S4:


,TADIG,MOC_NEW,MTC_NEW,SMS_NEW,DATA_NEW,SPONSOR_ID,LTE,CAMEL,NETWORK_ID,DailyCap_NEW,DailyCapLimit_NEW,is_discounted



Появившиеся тарифы S5:


,TADIG,MOC_NEW,MTC_NEW,SMS_NEW,DATA_NEW,SPONSOR_ID,LTE,CAMEL,NETWORK_ID,DailyCap_NEW,DailyCapLimit_NEW,is_discounted
3,AIAK9,0.5750,0.0575,0.05750,0.05750,NaN,No,NaN,NaN,0.0,NaN,NaN
4,ATGK9,0.5750,0.0575,0.05750,0.05750,NaN,No,NaN,NaN,0.0,NaN,NaN
5,BMUK9,0.2875,0.0345,0.02300,0.01150,NaN,No,NaN,NaN,0.0,NaN,NaN
6,BOLK9,NaN,0.0575,0.01150,0.11500,NaN,No,NaN,NaN,0.0,NaN,NaN
7,BFAK8,NaN,0.0000,NaN,0.01150,NaN,No,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
63,GINK9,0.3910,0.0000,0.01288,0.01288,NaN,No,NaN,NaN,0.0,NaN,NaN
64,GINK8,0.5750,0.0000,0.03450,0.01150,NaN,No,NaN,NaN,0.0,NaN,NaN
167,USAGU,0.5750,0.0000,0.01150,0.00575,NaN,Yes,NaN,1233.0,0.0,NaN,NaN
243,MACSM,1.2650,0.1725,0.11500,0.03450,NaN,No,NaN,568.0,0.0,NaN,NaN



Появившиеся тарифы S8:


,TADIG,MOC_NEW,MTC_NEW,SMS_NEW,DATA_NEW,SPONSOR_ID,LTE,CAMEL,NETWORK_ID,DailyCap_NEW,DailyCapLimit_NEW,is_discounted
3,AFGTD,0.8928,0.0000,0.3156,0.8880,NaN,No,No,7.0,0.0,NaN,NaN
22,AZEBC,1.2468,0.0000,0.2340,0.8880,NaN,Yes,Yes,58.0,0.0,NaN,NaN
23,BDIL1,0.8928,0.0000,0.2700,0.3552,NaN,No,No,NaN,0.0,NaN,NaN
24,BHSNC,1.3368,0.0000,0.1980,0.2136,NaN,Yes,Yes,NaN,0.0,NaN,NaN
25,BLZSC,0.8928,0.0900,0.1800,0.0900,NaN,Yes,Yes,NaN,0.0,NaN,NaN
26,CHNCM,0.4992,0.2496,0.1236,0.0144,NaN,Yes,No,NaN,0.0,NaN,NaN
27,GINAG,1.0164,0.0000,0.1236,0.1044,NaN,No,No,NaN,0.0,NaN,NaN
28,GNQHT,0.8028,0.0000,0.2700,0.2676,NaN,No,Yes,NaN,0.0,NaN,NaN
29,KIRKL,4.9632,0.0000,0.2544,0.1644,NaN,No,No,NaN,0.0,NaN,NaN
30,MCOM1,1.0164,0.0000,0.1332,0.2052,NaN,No,No,NaN,0.0,NaN,NaN


In [27]:
# Проверить пропавшие тарифы

for sponsor in sponsors:
    print(f'\nПропавшие тарифы {sponsor.name}:')
    display(sponsor.closed_tariffs(df_al_comb, df_tadig))


Пропавшие тарифы S1:


,COUNTRY,NETWORK,TADIG,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_COMB



Пропавшие тарифы S2:


,COUNTRY,NETWORK,TADIG,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_COMB
0,Monaco,Monaco Telecom S.A.M.,MCOM2,0.05175,0.0,0.023,0.046,616.0,S2



Пропавшие тарифы S4:


,COUNTRY,NETWORK,TADIG,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_COMB



Пропавшие тарифы S5:


,COUNTRY,NETWORK,TADIG,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_COMB



Пропавшие тарифы S8:


,COUNTRY,NETWORK,TADIG,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_COMB


In [28]:
# Проверать изменения в тарифах

for sponsor in sponsors:
    print(f'\nИзменения в тарифах {sponsor.name}:')
    display(sponsor.changes_in_tariffs_colorised())


Изменения в тарифах S1:


,TADIG,MOC_NEW,MOC_CURRENT,MTC_NEW,MTC_CURRENT,SMS_NEW,SMS_CURRENT,DATA_NEW,DATA_CURRENT,is_discounted,NETWORK_ID



Изменения в тарифах S2:


,TADIG,MOC_NEW,MOC_CURRENT,MTC_NEW,MTC_CURRENT,SMS_NEW,SMS_CURRENT,DATA_NEW,DATA_CURRENT,is_discounted,NETWORK_ID
27,KHMVC,0.57684,0.805,0,0,0.059685,0.0805,0.358225,0.0483,False,139
133,MACCT,nan,nan,0,0,nan,0.1632,0.0119025,0.0119025,False,565



Изменения в тарифах S4:


,TADIG,MOC_NEW,MOC_CURRENT,MTC_NEW,MTC_CURRENT,SMS_NEW,SMS_CURRENT,DATA_NEW,DATA_CURRENT,is_discounted,NETWORK_ID



Изменения в тарифах S5:


,TADIG,MOC_NEW,MOC_CURRENT,MTC_NEW,MTC_CURRENT,SMS_NEW,SMS_CURRENT,DATA_NEW,DATA_CURRENT,is_discounted,NETWORK_ID



Изменения в тарифах S8:


,TADIG,MOC_NEW,MOC_CURRENT,MTC_NEW,MTC_CURRENT,SMS_NEW,SMS_CURRENT,DATA_NEW,DATA_CURRENT,is_discounted,NETWORK_ID


In [29]:
# Сгенерировать файлы с различающимися и новыми тарифами для загрузки в OCS DB

for sponsor in sponsors:
    print(f'\nSponsor {sponsor.name}:')
    sponsor.prepare_file_to_close_tariffs()
    sponsor.prepare_file_to_add_different()
    sponsor.prepare_file_to_add_new()


Sponsor S1:

Sponsor S2:

Sponsor S4:

Sponsor S5:

Sponsor S8:


### Вспомогательные функции

In [ ]:
downloads

In [193]:
# Функция для очистки тарифов Smart и добавления CAMEL

smart_dirty_tariff_file = 'S5_20200101_FULL_DIRTY.csv'
smart_camel_file = 'CAMEL_PHLSR.csv'
smart_clean_tariff_file_smart_cost = 'S5_20200101_FULL_SMART_COST.csv'
smart_clean_tariff_file_rb_cost = 'S5_20200101_FULL_RB_COST.csv'

def clean_smart_file(folder, file):
    '''Функция для очистки тарифов Smart'''
    df_smart = pd.read_csv(join(folder, file), sep=',')
    try:
        df_smart.Data_Increment_KB = df_smart.Data_Increment_KB.str.lower().str.strip().replace({'1kb':1, '10kb':10})
    except:
        pass
    try:
        df_smart.LTE = df_smart.LTE.str.strip().replace({'3G':'No', '4G':'Yes'})
    except:
        pass
    df_smart.Data_Increment_KB.fillna(10, inplace=True)
    df_smart.Voice_Increment_SEC.fillna(60, inplace=True)
    df_smart.MTC.fillna(0, inplace=True)
    
    for col_name in ['MOC','MTC','SMS','DATA']:
        df_smart.loc[df_smart[col_name].str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(), col_name] =\
        df_smart[col_name].str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64') * eur_usd_rate
    
    df_smart[['MOC','MTC','SMS','DATA']] = df_smart[['MOC','MTC','SMS','DATA']].replace(to_replace={r'-':0}, regex=True)

    df_smart[['MOC','MTC','SMS','DATA']] = df_smart[['MOC','MTC','SMS','DATA']].astype('float64')
    
    return df_smart

def clean_add_camel(folder, file_camel, df_smart):
    '''Функция для добавления CAMEL'''
    camel_list = pd.read_csv(join(folder, file_camel)).TADIG.tolist()
    df_smart.loc[df_smart.TADIG.isin(camel_list), 'CAMEL'] = 'Yes'
    return df_smart

def to_csv_smart_cost_and_price(df_smart):
    df_smart.to_csv(join(downloads, smart_clean_tariff_file_smart_cost), index=False)
    df_smart[['MOC','MTC','SMS','DATA']] = df_smart[['MOC','MTC','SMS','DATA']] * smart_price_to_cost_rate
    df_smart.to_csv(join(downloads, smart_clean_tariff_file_rb_cost), index=False)

df_smart = clean_smart_file(downloads, smart_dirty_tariff_file)
df_smart = clean_add_camel(downloads, smart_camel_file, df_smart)
to_csv_smart_cost_and_price(df_smart)

df_smart.head()

,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCapLimit
0,AFGAW,NaN,0.1725,0.115000,0.115000,60,1.0,No,NaN,NaN
1,AFGEA,NaN,0.0000,0.017250,0.017250,60,1.0,No,NaN,NaN
2,ALBVF,0.161,0.0000,0.008625,0.008625,60,1.0,No,NaN,NaN
3,AIAK9,0.575,0.0575,0.057500,0.057500,60,1.0,No,NaN,NaN
4,JAMDC,NaN,0.0000,NaN,0.115000,60,10.0,No,Yes,NaN


In [6]:
# Конвертация EUR в USD для P4

tariff_file_name_eur = 'S2_20200601_FULL_EUR.csv'
tariff_file_name_usd = 'S2_20200601_FULL_USD.csv'

df_temp = pd.read_csv(join(downloads, tariff_file_name_eur))
df_temp[['MOC','MTC','SMS','DATA']] = df_temp[['MOC','MTC','SMS','DATA']] * s2_sponsor.eur_usd_rate
df_temp.to_csv(join(s2_sponsor.downloads, f'{s2_sponsor.tariff_file_name[:-7]}.csv'), index=False)
df_temp.head()

,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit
0,ALBAM,0.51635,0.0,0.05980,0.046000,1.0,1,Yes,Yes,NaN,NaN
1,ALBEM,0.44850,0.0,0.04600,0.044850,60.0,1,Yes,Yes,NaN,NaN
2,ARGTM,NaN,0.0,0.16445,0.046000,NaN,1,No,Yes,NaN,NaN
3,AUSOP,0.06325,0.0,0.00920,0.009200,60.0,1,Yes,Yes,NaN,NaN
4,AUTCA,0.02300,0.0,0.00345,0.001437,1.0,1,Yes,Yes,NaN,NaN


### Сделать

Добавить обработку CAMEL

### Developments